# Manipulate data with tensorflow

In this exercise you will practice manipulating tensors and forming tensor datasets with tensorflow.
We are taking advantage of this moment to let you only manipulate data as we will be focusing much more on building models in the following days!

In [1]:
import tensorflow as tf

## Practice tensor operations

* Create a constant tensor named `tensor1` containing the values `[0,1,2,3,4,5,6,7]` and a variable tensor named `tensor2` containing the values `[0,1,2,0,1,2]`

In [11]:
tensor1 = tf.constant([0,1,2,3,4,5,6,7])
tensor2 = tf.Variable([0,1,2,0,1,2])
print('tensor1:', tensor1)
print('tensor2:', tensor2)

tensor1: tf.Tensor([0 1 2 3 4 5 6 7], shape=(8,), dtype=int32)
tensor2: <tf.Variable 'Variable:0' shape=(6,) dtype=int32, numpy=array([0, 1, 2, 0, 1, 2], dtype=int32)>


* Reshape `tensor1` so it has 2 columns and 4 rows, and `tensor2` so it has 2 rows and 3 columns.
Has this operation changed the nature of `tensor2`? How could you change it back to its former nature?

In [12]:
tensor1 = tf.reshape(tensor1, [4,2])
tensor2 = tf.reshape(tensor2, [2,3])
print('tensor1:', tensor1)
print('tensor2:', tensor2)

tensor1: tf.Tensor(
[[0 1]
 [2 3]
 [4 5]
 [6 7]], shape=(4, 2), dtype=int32)
tensor2: tf.Tensor(
[[0 1 2]
 [0 1 2]], shape=(2, 3), dtype=int32)


In [13]:
tensor2 = tf.Variable(tensor2)
print('tensor2:', tensor2)

tensor2: <tf.Variable 'Variable:0' shape=(2, 3) dtype=int32, numpy=
array([[0, 1, 2],
       [0, 1, 2]], dtype=int32)>


* Use a tensorflow function to create `tensor3` with the same shape as `tensor2` but filled with 1

In [14]:
tensor3 = tf.ones_like(tensor2)
print('tensor3', tensor3)

tensor3 tf.Tensor(
[[1 1 1]
 [1 1 1]], shape=(2, 3), dtype=int32)


* Modify the value of `tensor2` by substracting the values in `tensor3`, use a method so that it is an in place operation. Why would not you be able to do that with `tensor1`?

In [15]:
tensor2.assign_sub(tensor3)
# we cannot do that with tensor1 because it is a constant tensor which is immutable

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=int32, numpy=
array([[-1,  0,  1],
       [-1,  0,  1]], dtype=int32)>

* Can you multiply `tensor1` and `tensor2` pointwise? How about with a matrix multiplication? Display the result of the possible operations.

In [16]:
tensor1*tensor2

InvalidArgumentError: ignored

In [17]:
tf.matmul(tensor1,tensor2)

<tf.Tensor: shape=(4, 3), dtype=int32, numpy=
array([[ -1,   0,   1],
       [ -5,   0,   5],
       [ -9,   0,   9],
       [-13,   0,  13]], dtype=int32)>

## Tabular data

This part of the exercise will let you deal with tabular data in order to make batch datasets ready to be fed to deep learning models.

* Using the `sklearn.datasets` module, load the mnist dataset thanks to the `load_digits` function.

In [19]:
from sklearn.datasets import load_digits
digits = load_digits()

* This function gives you a Data Bunch object, which works basically like a dictionnary. Create an object data containing the value of the `data` key and an object target containing the value of the `target` key.

In [21]:
data = digits.data
target = digits.target
print('data:', data)
print('target:', target)

data: [[ 0.  0.  5. ...  0.  0.  0.]
 [ 0.  0.  0. ... 10.  0.  0.]
 [ 0.  0.  0. ... 16.  9.  0.]
 ...
 [ 0.  0.  1. ...  6.  0.  0.]
 [ 0.  0.  2. ... 12.  0.  0.]
 [ 0.  0. 10. ... 12.  1.  0.]]
target: [0 1 2 ... 8 9 8]


* What is the shape of `data` and `target`? Can you understand what these objects represent using the `DESCR` key of the Data Bunch?

In [24]:
print('data shape:', data.shape)
print('target shape', target.shape)
print(digits.DESCR)

# data represents the images of handwritten digits in flattened mode, the 8x8 
# images are represented by 64 columns on each row of the object data

data shape: (1797, 64)
target shape (1797,)
.. _digits_dataset:

Optical recognition of handwritten digits dataset
--------------------------------------------------

**Data Set Characteristics:**

    :Number of Instances: 5620
    :Number of Attributes: 64
    :Attribute Information: 8x8 image of integer pixels in the range 0..16.
    :Missing Attribute Values: None
    :Creator: E. Alpaydin (alpaydin '@' boun.edu.tr)
    :Date: July; 1998

This is a copy of the test set of the UCI ML hand-written digits datasets
https://archive.ics.uci.edu/ml/datasets/Optical+Recognition+of+Handwritten+Digits

The data set contains images of hand-written digits: 10 classes where
each class refers to a digit.

Preprocessing programs made available by NIST were used to extract
normalized bitmaps of handwritten digits from a preprinted form. From a
total of 43 people, 30 contributed to the training set and different 13
to the test set. 32x32 bitmaps are divided into nonoverlapping blocks of
4x4 and the

* Can you visualize the first image in data ?

In [28]:
from plotly import express as px
first_image = data[0,:].reshape([8,8])
px.imshow(first_image,  color_continuous_scale='gray')

* The pixel values from those images is encoded in integers between 0 and 255, it is always better to feed your deep learning models with reasonnably scaled data to avoid the network not being able to learn. To do this we'll divide the value in each pixel by 255. Do this.

In [35]:
data = data/255

### Technique 1: Split the data with sklearn

Most of the time when you will be dealing with data you want to feed to a deap learning model, you will have a pandas DataFrame or numpy array at some points that contains some representation of your data and the associated values of the target variable. In those cases, it's easier to just split the data in a train and validation set using sklearn. (Remember that for very large datasets or for training and evaluating deep learning models we most of the time use the three way hold out method, where on set serves as the training set, one as the validation set to control for overfitting, and the last one is the test set against which we will evaluate the model).

* Split the data and target into three different parts, one containing the train set (60%), another with the validation set (20%), and a third with the test set (20%), using sklearn.

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_valtest, y_train, y_valtest = train_test_split(data,target, test_size=0.4)
X_val, X_test, y_val, y_test = train_test_split(X_valtest, y_valtest, test_size=0.5)
print("X_train", X_train.shape, "y_train", y_train.shape)
print("X_val", X_val.shape, "y_val", y_val.shape)
print("X_test", X_test.shape, "y_test", y_test.shape)

X_train (1078, 64) y_train (1078,)
X_val (359, 64) y_val (359,)
X_test (360, 64) y_test (360,)


* Form three tensor slice datasets using the training validation and test data.

In [37]:
train = tf.data.Dataset.from_tensor_slices((X_train,y_train))
val = tf.data.Dataset.from_tensor_slices((X_val,y_val))
test = tf.data.Dataset.from_tensor_slices((X_test,y_test))
print("train:",train)
print("val:", val)
print("test:", test)

train: <TensorSliceDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)>
val: <TensorSliceDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)>
test: <TensorSliceDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)>


* Shuffle these tensor slice datasets and arrange them in batches of 8 observations, then display one batch from each of these batch datasets.

In [38]:
train_batch = train.shuffle(len(X_train)).batch(8)
val_batch = train.shuffle(len(X_val)).batch(8)
test_batch = train.shuffle(len(X_test)).batch(8)

print("train batch:", next(iter(train_batch)))
print("val batch:", next(iter(val_batch)))
print("test batch:", next(iter(test_batch)))

train batch: (<tf.Tensor: shape=(8, 64), dtype=float64, numpy=
array([[0.        , 0.        , 0.01960784, 0.05882353, 0.05098039,
        0.01176471, 0.        , 0.        , 0.        , 0.        ,
        0.04705882, 0.05882353, 0.04705882, 0.05490196, 0.        ,
        0.        , 0.        , 0.        , 0.04705882, 0.04705882,
        0.05490196, 0.05490196, 0.        , 0.        , 0.        ,
        0.        , 0.01176471, 0.0627451 , 0.05490196, 0.01176471,
        0.        , 0.        , 0.        , 0.        , 0.03529412,
        0.05490196, 0.05882353, 0.01176471, 0.        , 0.        ,
        0.        , 0.00392157, 0.05882353, 0.01960784, 0.03137255,
        0.04705882, 0.00392157, 0.        , 0.        , 0.        ,
        0.0627451 , 0.01568627, 0.01568627, 0.0627451 , 0.01568627,
        0.        , 0.        , 0.        , 0.01960784, 0.0627451 ,
        0.0627451 , 0.04705882, 0.00784314, 0.        ],
       [0.        , 0.00392157, 0.04705882, 0.0627451 , 0.039215

We are now ready to start training deep learning models!

## Technique 2: split using tensorflow

This technique is not so recommended because tensorflow is not able to work with datasets in the same way that sklearn does, it is not as practical to split the data in a random way, but we will show you how it can be done, as sometimes you will strictly be working with tensorflow objects.

* Create a tensor slice dataset object using `data`and `target`.

In [40]:
full_ds = tf.data.Dataset.from_tensor_slices((data,target))
print("full_ds:", full_ds)

full_ds: <TensorSliceDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)>


* Using the commands take and skip, separate the tensor slice dataset into a train object containing 60% of the data, a val object (20%) and a test object (20%).

In [45]:
n_train = int(0.6*len(data))
n_val = int(0.2*len(data))
n_test = len(data) - n_train - n_val

train = full_ds.take(n_train)
valtest = full_ds.skip(n_train)
val = valtest.take(n_val)
test = valtest.skip(n_val)

print("train:", train, len(train))
print("val:", val, len(val))
print("test:", test, len(test))

train: <TakeDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)> 1078
val: <TakeDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)> 359
test: <SkipDataset shapes: ((64,), ()), types: (tf.float64, tf.int64)> 360


* Use methods shuffle and batch in order to create batch datasets with batches of 8 observations for train, val, and test, and show one batch from each of these objects.

In [46]:
train_batch = train.shuffle(n_train).batch(8)
val_batch = val.shuffle(n_val).batch(8)
test_batch = test.shuffle(n_test).batch(8)

print("train batch:", next(iter(train_batch)))
print("val batch:", next(iter(val_batch)))
print("test batch:", next(iter(test_batch)))

train batch: (<tf.Tensor: shape=(8, 64), dtype=float64, numpy=
array([[0.        , 0.        , 0.00784314, 0.05490196, 0.01960784,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.03529412, 0.04705882, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.00392157, 0.05882353, 0.00392157,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.01176471, 0.05882353, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.02352941, 0.0627451 ,
        0.0627451 , 0.0627451 , 0.05098039, 0.00392157, 0.        ,
        0.        , 0.00784314, 0.0627451 , 0.03137255, 0.01568627,
        0.02745098, 0.04313725, 0.        , 0.        , 0.        ,
        0.04705882, 0.04313725, 0.00392157, 0.03137255, 0.04313725,
        0.        , 0.        , 0.        , 0.01176471, 0.04705882,
        0.0627451 , 0.05882353, 0.01568627, 0.        ],
       [0.        , 0.        , 0.00392157, 0.05098039, 0.047058

Congratulations, you know two different ways of forming datasets that are fit for training deep learning models with tensorflow! This skill will come in very handy as we will try to focus more on building models from now on, and put less focus on preprocessing.
Until then, happy learning!